<a href="https://colab.research.google.com/github/iris-baptista/daa_seamCarving/blob/main/G20_111812_EurisaPatricio_122701_IrisBaptista.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Minitrabalho DAA - "*Seam Carving*"
# Ano Letivo 2024/2025

## 1. Bibliotecas Importadas

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from PIL import Image

## 2. Representação de Dados

### 2.1 Como está a representar a energia dos pixels no seu grafo? Qual foi o critério para esta escolha? Que tipo de grafo representa o problema em questão?

**Optou-se por representar a energia dos pixels como peso das arestas no grafo. **

Os **nós/vértices** estarão associados aos pixels da imagem, e o **peso das arestas** corresponderão à energia de cada pixel **sucessor/destino**, que neste caso será calculada com base na combinação linear do valor da intensidade dos pixels na sua vizinhança inferior, conforme mencionado no enunciado. Contudo, sabe-se que o grafo será um digrafo, pesado e acíclico, ou seja um DAG, O que será útil para encontrarmos o caminho mais curto, ou seja, com menor energia, e, posteriormente, fazermos a sua remoção.

### 2.2 Qual é a representação computacional de grafo que está a utilizar? Por exemplo, matriz de adjacência, lista/mapa de adjacências ou uma outra alternativa?

A representação computacional do grafo que iremos utilizar é o **Mapa de Adjacências**, um dicionário onde cada pixel terá como valor outro dicionário que apontará para os seus vizinhos inferiores, sendo o respetivo valor de cada vizinho uma aresta (tendo a sua respetiva energia como peso).

### 2.3 Identifique as vantagens e desvantagens da sua representação de grafo escolhida e os critérios utilizados para a sua escolha. Por exemplo, a sua escolha facilita a implementação de alguma operação específica? Ou faz com que as operações fiquem mais eficientes (em relação ao tempo e ao espaço em memória)?

Neste problema, identificamos que o grafo será esparso. Ele terá aproximadamente W × H vértices e, em média, m ≈ 3WH arestas, pois cada vértice pode se conectar a no máximo 3 vizinhos inferiores.
Assim, a densidade do grafo será:

                             m / n² ≈ 3 / (W × H)

Como este valor é muito baixo e m « n², podemos concluir que é  mesmo um grafo esparso.
Sendo assim vantajoso usar um mapa de adjacências.


**Vantagens:**
- Melhor uso da memória (O(n + m)).
- Verificação e inserção de arestas em tempo O(1).
- Permite associar informações (como energia) diretamente às arestas.

**Desvantagens:**
- Menos eficiente em grafos densos (o que não é o caso).


### 2.4 Se optou por utilizar as classes Graph/Digraph fornecidas na Semana 6 para a sua solução, identifique também as possíveis modificações que teve de realizar nas classes.

*Responder*...

## 3. API Seam Carving

### 3.1 Estrutura de Dados

In [ ]:

class Vertex:
    def __init__(self, vertex_id):
        self._vertex_id = vertex_id

    def __hash__(self):
        return hash(self._vertex_id)

    def __str__(self):
        return str(self._vertex_id)

    def vertex_id(self):
        return self._vertex_id

class Edge:
    def __init__(self, u, v, weight=0):
        self._vertex_1 = u
        self._vertex_2 = v
        self._weight = weight

    def __hash__(self):
        # Função que mapeia a aresta a uma posição no dicionário (hash map)
        return hash( (self._vertex_1, self._vertex_2) )

    def __eq__(self, other):
        # define igualdade de duas arestas (deve ser consistente com a função hash)
        return self._vertex_1 == other._vertex_1 and self._vertex_2 == other._vertex_2

    def endpoints(self):
        return (self._vertex_1, self._vertex_2)

    def cost(self):
        return self._weight

class Graph:

    def __init__(self):
        self._adjancencies = {}
        self._vertices = {}
        self._n = 0
        self._m = 0

    def insert_vertex(self, vertex_id):
        '''Insere um novo vértice com o id vertex_id.'''
        if not self.has_vertex(vertex_id):
            vertex = Vertex(vertex_id)          # instancia um objeto do tipo Vertex
            self._vertices[vertex_id] = vertex  # insere o novo vértice no dicionário de vertices
            self._adjancencies[vertex] = {}     # inicializa o mapa de adjacências deste vértice a vazio
            self._n +=1                         # mais um vértice no grafo

    def insert_edge(self, u_id, v_id, weight=0):
        ''' Cria e insere uma nova aresta entre u_id e v_id com peso weight.
            Se a aresta já existe no grafo, atualiza-se o seu peso.
            Também insere os vértices u_id e v_id, caso não existam.'''
        if not self.has_vertex(u_id):
            self.insert_vertex(u_id) # insere novo vertex e atualiza n
        if not self.has_vertex(v_id):
            self.insert_vertex(v_id) # insere novo vertex e atualiza n
        if not self.has_edge(u_id, v_id):
            self._m +=1           # atualiza m apenas se a aresta ainda não existir no grafo
        else:
            print(f"Existing edge {u_id} and {v_id}. Will only update weight")
        vertex_u = self._vertices[u_id]  # acede ao objeto Vertex associado a u_id
        vertex_v = self._vertices[v_id]  # acede ao objeto Vertex associado a v_id
        e = Edge(vertex_u, vertex_v, weight)
        self._adjancencies[vertex_u][vertex_v] = e  # coloca v nas adjacências de u
   #     self._adjancencies[vertex_v][vertex_u] = e  # e u nas adjacências de v (para facilitar a procura de todas as arestas incidentes num vértice)

    def get_vertex(self, vertex_id):
        return self._vertices[vertex_id]

    def get_edge(self, u_id, v_id):
        ''' Devolve o objeto aresta (Edge) que liga u_id a v_id.
            Devolve None se não forem adjacentes ou se (um d)os vértices não existirem.'''
        if not self.has_edge(u_id, v_id):
            return None
        else:
            vertex_u = self._vertices[u_id]
            vertex_v = self._vertices[v_id]
            return self._adjancencies[vertex_u][vertex_v]

    def vertices(self):
        '''Devolve um iterável sobre todos os vértices do Grafo (tipo Vertex)'''
        return self._vertices.values()

    def edges(self):
        '''Devolve um iterável sobre todas as arestas do Grafo (sem arestas duplicadas).'''
        seen = {}      # evita a repetição de arestas no grafo não orientado
        for adj_map in self._adjancencies.values():
            for edge in adj_map.values():
                if edge not in seen:
                    yield edge
                seen[edge] = True

    def incident_edges(self, vertex_id):
        '''Devolve um iterável (gerador) com todas as arestas de um vértice com id vertex_id.'''
        vertex = self._vertices[vertex_id]
        for edge in self._adjancencies[vertex].values(): # para todas as arestas incidentes em v:
            yield edge

    def has_neighbors(self, vertex_id):
        '''Verifica se o vértice de id vertex_id tem vértices adjacentes (vizinhos).'''
        if not self.has_vertex(vertex_id):
            return False
        return self.degree(vertex_id) == 0

    def remove_vertex(self, vertex_id):
        '''Remove o vértice com id vertex_id. Se o vértice não existir, não faz nada.'''
        # Passo 1: remover todas as arestas do vértice dado
        # Passo 2: remover todas as arestas incidentes em vertex_id dos mapas de outros vertices
        # Passo 3: remover o vértice com id vertex_id do grafo
        # Passo 4: decrementa contador de vértices
        if self.has_vertex(vertex_id):
            lst_copied = list(self.incident_edges(vertex_id)) # copia para a lista para evitar erros de concorrência (remove enquanto itera na lista)
            for edge in lst_copied:
                x, y = edge.endpoints()
                self.remove_edge(x.vertex_id(),y.vertex_id())  # (Passos 1 e 2)
            del self._adjancencies[self._vertices[vertex_id]]  # (Passo 3 - remove do dicionário de adjacências)
            del self._vertices[vertex_id]                      # (Passo 3 - remove do dicionário de vértices)
            self._n -=1                                        # (Passo 4 - decrementa contador)

    def remove_edge(self, u_id, v_id):
        '''Remove a aresta entre u_id e v_id. Se a aresta não existir, não faz nada.'''
        if  self.has_edge(u_id, v_id):
            vertex_u = self._vertices[u_id]
            vertex_v = self._vertices[v_id]
            del self._adjancencies[vertex_u][vertex_v]
            if vertex_u != vertex_v:  # laços são removidos apenas uma vez
                del self._adjancencies[vertex_v][vertex_u]
            self._m -=1

class DiGraph(Graph):
    def __init__(self):
        '''Construtor: Cria um grafo vazio (dicionário de _adjancencies).'''
        super().__init__()
        self._in_adjancencies = {}     # dicionário com chave vértice e valor mapa de adjacências das arestas de entrada

    def is_directed(self):
        '''A classe DiGraph representa um grafo orientado.'''
        return True

    def __repr__(self):
        '''Devolve a representação completa do grafo em string (debug).'''
        if self._n == 0:
            ret = "DAA-DiGraph out-Representation: <empty>\n"
            ret += "DAA-DiGraph in-Representation: <empty>\n"
        else:
            ret = "DAA-DiGraph out-Representation:\n"
            for vertex in self._adjancencies.keys():
                ret += str(vertex)
                ret += " out-deg " + str(self.out_degree(vertex.vertex_id())) + ":\t"
                for edge in self.successors(vertex.vertex_id()):
                    ret += str(edge) + "; "
                ret += "\n"
            ret += "DAA-DiGraph in-Representation:\n"
            for vertex in self._in_adjancencies.keys():
                ret += str(vertex)
                ret += " in-deg " + str(self.in_degree(vertex.vertex_id())) + ":\t"
                for edge in self.predecessors(vertex.vertex_id()):
                    ret += str(edge) + "; "
                ret += "\n"
        return ret

    def __str__(self):
        '''Devolve a representação do grafo em string.'''
        if self._n == 0:
            ret = "DAA-DiGraph: <empty>\n"
        else:
            ret = "DAA-Graph:\n"
            for vertex in self._adjancencies.keys():
                #ret += "vertex-"
                ret += str(vertex) + ": "
                for edge in self.successors(vertex.vertex_id()):
                    ret += str(edge) + "; "
                ret += "\n"
        return ret

    def is_successor(self, u_id, v_id):
        """Verifica se o nó u é sucessor do vértice v no grafo."""
        return super().has_edge(v_id, u_id)

    def is_predecessor(self, u_id, v_id):
        """ Verifica se o vértice u é antecessor do vértice v no grafo.
            Corresponde ao método super().has_edge(u,v). """
        return super().has_edge(u_id, v_id)

    def insert_vertex(self, vertex_id):
        '''Insere um novo nó com id vertex_id.'''
        super().insert_vertex(vertex_id)   # inicializa o mapa dos arcos de saída (_adjancencies)
        vertex = self._vertices[vertex_id]
        self._in_adjancencies[vertex] = {}  # inicializa o mapa dos arcos de entrada (_in_adjancencies)

    def insert_edge(self, u_id, v_id, weight=0):
        ''' Cria e insere um novo arco entre u_id e v_id com peso weight.
            Se o arco já existe no grafo, atualiza-se o seu peso.
            Também insere os nós u_id e v_id, caso não existam.'''
        if not self.has_vertex(u_id):
            self.insert_vertex(u_id) # insere novo vertex e atualiza n
        if not self.has_vertex(v_id):
            self.insert_vertex(v_id) # insere novo vertex e atualiza n
        if not self.has_edge(u_id, v_id):
            self._m +=1           # atualiza m apenas se a aresta ainda não existir no grafo
        else:
            print(f"Existing edge {u_id} and {v_id}. Will only update weight")
        vertex_u = self._vertices[u_id]
        vertex_v = self._vertices[v_id]
        e = Edge(vertex_u, vertex_v, weight)
        self._adjancencies[vertex_u][vertex_v] = e  # coloca v nas adjacências de u
    #    self._in_adjancencies[vertex_v][vertex_u] = e  # e u nas adjacências de v (para facilitar a procura de todas as arestas incidentes num vértice)

    def out_degree(self, vertex_id):
        '''Quantidade de arcos que saem do nó v.
        '''
        return len(self._adjancencies[self._vertices[vertex_id]]) # Verifica o tamanho do mapa de saída de v

    def in_degree(self, vertex_id):
        '''Quantidade de arcos que entram no nó v.
        '''
        return len(self._in_adjancencies[self._vertices[vertex_id]]) # Verifica o tamanho do mapa de entrade de v

    def edges(self):
        '''Devolve um iterável sobre todos os arcos do Grafo.'''
        for adj_map in self._adjancencies.values():
            for edge in adj_map.values():
                yield edge

    def successors(self, vertex_id): # igual ao super().incident_edges(v)
        '''Devolve um iterável com todos os arcos que saem de v.'''
        return self.incident_edges(vertex_id)
        '''
        vertex = self._vertices[vertex_id]
        for edge in self._adjancencies[vertex].values(): # para todas os arcos que saem do vértice
            yield edge
            '''

    def predecessors(self, vertex_id):
        '''Devolve um iterável com todos os arcos que entram em v.'''
        vertex = self._vertices[vertex_id]
        for edge in self._in_adjancencies[vertex].values(): # para todas os arcos que entram no vértice
            yield edge

    def remove_edge(self, u_id, v_id):
        '''Remove o arco u_id -> v_id. Se o arco não existir, não faz nada.'''
        if  self.has_edge(u_id, v_id):
            vertex_u = self._vertices[u_id]
            vertex_v = self._vertices[v_id]
            del self._adjancencies[vertex_u][vertex_v]
            del self._in_adjancencies[vertex_v][vertex_u]
            self._m -=1

    def remove_vertex(self, vertex_id):
        '''Remove o nó com id vertex_id. Se o nó não existir, não faz nada.'''
        # Passo 1: remover todos os arcos que saem do vértice dado
        # Passo 2: remover todos os arcos que entram no vértice dado
        # Passo 3: remover o vértice do grafo
        if self.has_vertex(vertex_id):
            lst_out = list(self.successors(vertex_id)) # copia para a lista para evitar erros de concorrência
            for edge in lst_out:
                x, y = edge.endpoints()
                self.remove_edge(x.vertex_id(),y.vertex_id())       # (Passos 1)
            lst_in = list(self.predecessors(vertex_id)) # copia para a lista para evitar erros de concorrência
            for edge in lst_in:
                x, y = edge.endpoints()
                self.remove_edge(x.vertex_id(),y.vertex_id())       # (Passos 2)
            del self._adjancencies[self._vertices[vertex_id]]     # (Passo 3 - remove do dicionário de saídas)
            del self._in_adjancencies[self._vertices[vertex_id]]  # (Passo 3 - remove do dicionário de entradas)
            del self._vertices[vertex_id]                         # (Passo 3 - remove do dicionário de vértices)
            self._n -=1


In [ ]:
import numpy as np
from collections import defaultdict

class SeamCarving:
    def __init__(self, image):
        self.image = image.astype(np.int32)
        self.height, self.width = self.image.shape[:2]
        self.energy_map = self._calculate_energy()

    def _calculate_energy(self):
        return

    def find_vertical_steam(self):
        g = DiGraph()
        h, w = self.height, self.width
        energy = self.energy_map

        # Criar vértices
        for y in range(h):
            for x in range(w):
                g.insert_vertex((x, y))

        # Criar arestas de cima para baixo
        for y in range(h - 1):
            for x in range(w):
                u = (x, y)
                for dx in [-1, 0, 1]:
                    nx = x + dx
                    ny = y + 1
                    if 0 <= nx < w:
                        v = (nx, ny)
                        g.insert_edge(u, v, weight=energy[ny, nx])

        # Fonte e destino virtuais
        source = "SOURCE"
        target = "TARGET"
        g.insert_vertex(source)
        g.insert_vertex(target)

        for x in range(w):
            g.insert_edge(source, (x, 0), weight=energy[0, x])
            g.insert_edge((x, h - 1), target, weight=0)

        # Bellman-Ford
        dist = defaultdict(lambda: float('inf'))
        pred = {}
        dist[g.get_vertex(source)] = 0

        for _ in range(len(g._vertices) - 1):
            for u in g._adjancencies:
                for v in g._adjancencies[u]:
                    e = g._adjancencies[u][v]
                    cost = e.cost()
                    if dist[u] + cost < dist[v]:
                        dist[v] = dist[u] + cost
                        pred[v] = u

        # Reconstruir o caminho da costura
        path = []
        v = g.get_vertex(target)
        while v != g.get_vertex(source):
            v_id = v.vertex_id()
            if v_id not in [source, target]:
                path.append(v_id)
            v = pred[v]

        return list(reversed(path))

    def remove_vertical_steam(self, steam):
        h, w = self.height, self.width
        new_image = np.zeros((h, w - 1, 3), dtype=self.image.dtype)

        for y in range(h):
            x = steam[y][0]  # steam é uma lista de (x, y)
            new_image[y, :, :] = np.delete(self.image[y, :, :], x, axis=0)

        self.image = new_image
        self.height, self.width = self.image.shape[:2]
        self.energy_map = self._calculate_energy()

    def picture(self):
        return


### 3.2 Calcular a energia da imagem

3.2.1 Implemente e teste a função auxiliar _calculate_energy() que irá calcular a energia da imagem atual. A função deve devolver o mapa da energia da imagem como um ndarray (numpy array).

In [ ]:
def _calculate_energy(self):
    h, w = self.height, self.width
    energy_map = np.zeros((h, w), dtype=np.float32)
    for y in range(h):
        for x in range(w):
            if x == 0 or x == w - 1 or y == 0:
                energy_map[y, x] = 0
                continue
            dx = abs(self.image[y, x + 1, 0] - self.image[y, x - 1, 0])
            dy = abs(self.image[y + 1, x, 0] - self.image[y - 1, x, 0])
            energy_map[y, x] = dx + dy
    return energy_map

3.2.2 Apresente uma análise da complexidade desta função em relação ao tempo e ao espaço extra de memória utilizados (em função da dimensão da imagem).

Responder...

### 3.3 Encontrar a costura de menor energia (steam).

3.3.1 Descreva os passos e as modificações realizadas no grafo e/ou no algoritmo para reduzir o problema do seam carving para o problema do caminho mais curto num grafo (shortest path). Indique qual algoritmo shortest path que escolheu implementar e o motivo da sua escolha.

Resposta...

3.3.2 Implemente e teste o método find_vertical_steam()que deverá encontrar o caminho de menor energia.

In [ ]:
steam = sc.find_vertical_steam()
print("Pixels da costura de menor energia:")
print(steam[:5], "...")  # Mostrar primeiros 5 pontos
print("Tamanho da costura:", len(steam))

# Visualizar costura sobre a imagem original
img_with_steam = sc.image.copy()
for x, y in steam:
    img_with_steam[y, x] = [255, 0, 0]  # marca a seam a vermelho

plt.imshow(img_with_steam.astype(np.uint8))
plt.title("Imagem com costura marcada")
plt.show()

3.3.3 Apresente uma análise completa da complexidade do seu algoritmo em função da dimensão da imagem, e compare-a com a complexidade de outras versões do algoritmo shortest path.

3.3.4 Analise e compare a complexidade do seu algoritmo baseado em grafos com uma solução que utiliza a estratégia de programação dinâmica. Na sua pesquisa bibliográfica e análise, deve usar fontes credíveis, tais como, artigos científicos, manuais técnicos ou recursos webgráficos devidamente acreditados.

### 3.4 Remover uma costura (steam) da imagem.

3.4.1 Implemente e teste o método remove_vertical_steam(steam) que recebe uma coleção com a sequência de pixels da costura, steam, e remove-os da imagem atual.

In [ ]:
original_shape = sc.image.shape
print("Antes da remoção:", original_shape)

sc.remove_vertical_steam(steam)

new_shape = sc.image.shape
print("Após remoção:", new_shape)

plt.imshow(sc.picture())
plt.title("Imagem após remover a costura")
plt.show()


3.4.2 Apresente uma análise da complexidade desta operação em função da dimensão da imagem.

## 4. Validação

### 4.1 Crie uma função que receba uma imagem e um fator de escala como entrada, e que devolva a imagem redimensionada utilizando a API SeamCarving. A função deve ser capaz de lidar com o redimensionamento da largura e da altura de uma imagem, consoante a escolha do utilizador.

### 4.2 Reduza a largura da imagem img-broadway_tower.jpg para 70% da sua largura original, utilizando a função implementada. O resultado deve ser apresentado no próprio notebook.

### 4.3 Reduza a largura da imagem img-brent-cox-unsplash.jpg para 60% da sua altura original, utilizando a função implementada. O resultado deve ser apresentado no próprio notebook.

## 5. Questões Éticas

### a) Se colaborou com alguém fora do seu grupo, indique aqui os respetivos nomes.

Não.

### b) Deve citar todas as fontes utilizadas fora do material da UC.